<a href="https://colab.research.google.com/github/Carabinero/politeness-review-classification/blob/main/C%C3%B3pia_de_classif_texto_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importação de biblioetcas e dados

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from string import punctuation
import re
from transformers import pipeline, AutoTokenizer, AutoModel
from gensim import utils
import gensim.models
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, classification_report, confusion_matrix
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

PolitenessDF = pd.read_csv('PolitenessDataset-FULL.csv')
print('Data Stats:', PolitenessDF.describe())
PolitenessDF.head(10)

Data Stats:               Tone
count  2500.000000
mean      2.867200
std       0.948854
min       1.000000
25%       2.000000
50%       3.000000
75%       3.000000
max       5.000000


,Venue,Review ID,review,Tone,Review URL
0,ShitMyReviewerSay,NaN,"It is early in the year, but difficult to imag...",2,NaN
1,ShitMyReviewerSay,NaN,You do not use the empirical data for the anal...,2,NaN
2,ShitMyReviewerSay,NaN,I understand that Wikipedia is not the best so...,3,NaN
3,ShitMyReviewerSay,NaN,Reviewer #1: 'The project can hardly be descri...,3,NaN
4,ShitMyReviewerSay,NaN,The figures are dishonest and not all that use...,2,NaN
5,ShitMyReviewerSay,NaN,Find your inner nerdâ€”it must be a big part o...,1,NaN
6,ShitMyReviewerSay,NaN,[entire review] 'Research method is very impor...,4,NaN
7,ShitMyReviewerSay,NaN,Some papers are a pleasure to read. This is no...,2,NaN
8,ShitMyReviewerSay,NaN,"Sorry guys, I'm throwing in the towel.",1,NaN
9,ShitMyReviewerSay,NaN,Nobody in their right mind would ever suggest ...,2,NaN


In [ ]:
# Set up the LabelEncoder object
# enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
# PolitenessDF["Tone_enc"] = enc.fit_transform(PolitenessDF["Tone"])

# Compare the two columns
# print(PolitenessDF.head())
Tone_enc = pd.get_dummies(PolitenessDF["Tone"])
print(Tone_enc.head())


       1      2      3      4      5
0  False   True  False  False  False
1  False   True  False  False  False
2  False  False   True  False  False
3  False  False   True  False  False
4  False   True  False  False  False


Pre-processamento dos dados

In [ ]:
print('Special Characters:', punctuation)

# stop_words = stopwords.words('portuguese')
def preprocess(x: str):
    new_x = x.replace(r'"',' ')
    for c in punctuation:
        new_x = new_x.replace(c,' ')
    pattern = r"""
    [^\w\s]|         # Remove punctuation
    http\S+|         # Remove links
    @\w+|            # Remove mentions
    #\S+|            # Remove hashtags
    \b\w*\d\w*\b|    # Remove words containing numbers
    \s+              # Normalize spaces
    """
    new_x = re.sub(pattern, ' ', new_x, flags=re.VERBOSE) #removendo pontuação do texto
    return new_x.lower().strip()


## Pré-processar datasets

df = PolitenessDF.copy()
df_filt = df.drop(["Venue", "Review ID", "Review URL"], axis=1)
print(df_filt.head())
df_filt = df_filt.dropna(subset=["review"])
print(df_filt.isna().sum())

df_filt['review_pro'] = df_filt['review'].apply(preprocess)

tfidf_vec = TfidfVectorizer()

# Transform the text into tf-idf vectors
text_tfidf = tfidf_vec.fit_transform(df_filt["review_pro"])

print(text_tfidf.shape)
print(df_filt.shape)
print(df_filt.value_counts("Tone"))

Special Characters: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
                                              review  Tone
0  It is early in the year, but difficult to imag...     2
1  You do not use the empirical data for the anal...     2
2  I understand that Wikipedia is not the best so...     3
3  Reviewer #1: 'The project can hardly be descri...     3
4  The figures are dishonest and not all that use...     2
review    0
Tone      0
dtype: int64
(2499, 5425)
(2499, 3)
Tone
3    1141
2     581
4     465
1     210
5     102
Name: count, dtype: int64


<ipython-input-105-062f6133f8d4>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt['review_pro'] = df_filt['review'].apply(preprocess)


treinamento do modelo

In [ ]:
# Split the dataset according to the class distribution of category_desc
tone_set = pd.get_dummies(df_filt["Tone"])
y=tone_set[5]
X=text_tfidf.toarray()
# print(df_filt.shape)
# df = pd.concat([df, tone_set], axis=1)
# df.head()
# print(text_tfidf.toarray().shape)
# X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(), df_filt["Tone"], test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)

print(y.value_counts())
print(y_train.value_counts())
print(y_test.value_counts())

nb = GaussianNB()
# Fit the model to the training data
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

# Avaliar modelo
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Print out the model's accuracy
print(nb.score(X_test, y_test))

knn= KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

linear_regression = LogisticRegression()
linear_regression.fit(X_train, y_train)
print(linear_regression.score(X_test, y_test))



5
False    2397
True      102
Name: count, dtype: int64
5
False    1606
True       68
Name: count, dtype: int64
5
False    791
True      34
Name: count, dtype: int64
Matriz de Confusão:
[[747  44]
 [ 32   2]]

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.96      0.94      0.95       791
        True       0.04      0.06      0.05        34

    accuracy                           0.91       825
   macro avg       0.50      0.50      0.50       825
weighted avg       0.92      0.91      0.91       825

0.9078787878787878
0.96
0.9587878787878787


avaliação dos resultados

In [ ]:
# prompt: create a bag of words of df_filt["review"] to use it to train a GaussianNB model

from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer to the preprocessed reviews
vectorizer.fit(df_filt["review_pro"])

# Transform the reviews into a bag-of-words representation
bag_of_words = vectorizer.transform(df_filt["review_pro"])

# Now you can use bag_of_words to train your GaussianNB model
# For example:
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df_filt["Tone"], test_size=0.33, random_state=42)

nb = GaussianNB()
nb.fit(X_train.toarray(), y_train) # GaussianNB requires a dense matrix
print(nb.score(X_test.toarray(), y_test))

0.3660606060606061


In [ ]:
# prompt: create a bag of words of df_filt["review"] to use it to train a GaussianNB model using bernouli distance and a multinomial one
# !pip install sklearn.neighbors
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# from sklearn.neighbors import DistanceMetric

# Assuming df_filt and other necessary variables are defined as in the previous code

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer to the preprocessed reviews
vectorizer.fit(df_filt["review_pro"])

# Transform the reviews into a bag-of-words representation
bag_of_words = vectorizer.transform(df_filt["review_pro"])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df_filt["Tone"], test_size=0.33, random_state=42)


# GaussianNB with default distance metric
gnb = GaussianNB()
gnb.fit(X_train.toarray(), y_train)
y_pred_gnb = gnb.predict(X_test.toarray())
print(f"GaussianNB Accuracy (default): {accuracy_score(y_test, y_pred_gnb)}")

# Define a custom distance metric for BernoulliNB
# bernoulli_dist = DistanceMetric.get_metric('jaccard')

# BernoulliNB - we can't directly use a distance metric, instead we use the jaccard similarity
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
y_pred_bnb = bnb.predict(X_test)
print(f"BernoulliNB Accuracy: {accuracy_score(y_test, y_pred_bnb)}")

# MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_mnb = mnb.predict(X_test)
print(f"MultinomialNB Accuracy: {accuracy_score(y_test, y_pred_mnb)}")

# RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f"RandomForest Accuracy: {accuracy_score(y_test, y_pred_rf)}")

#

GaussianNB Accuracy (default): 0.3660606060606061
BernoulliNB Accuracy: 0.4896969696969697
MultinomialNB Accuracy: 0.48727272727272725
RandomForest Accuracy: 0.5248484848484849
